In [1]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1Hydbg7KXOoK9Xf-DPFsoMGv6Whdzhh1F'})
download.GetContentFile('train_full.csv')
download = drive.CreateFile({'id': '1uo-5rbDJKUSBondu2NnaMC4ygu1TwuW0'})
download.GetContentFile('test_full.csv')

In [0]:
import pandas as pd
import numpy as np

label_name = 'non-toxic'

train_full = pd.read_csv('train_full.csv')
test_full = pd.read_csv('test_full.csv')



In [5]:
more_rows = test_full.iloc[0:4000,:]
test_full = test_full.drop(test_full.index[0:4000])

train_full = pd.concat([train_full,more_rows],axis='rows',ignore_index=True).reset_index(drop=True)
print(train_full.shape,test_full.shape)

(23993, 9) (6998, 9)


Create word vectors

In [0]:
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report

In [0]:
train = train_full["comment_text"]
test = test_full["comment_text"]

trainlab = train_full[label_name]
testlab = test_full[label_name]

In [8]:
# string cleaning- already done
# Download and load the punkt tokenizer for sentence splitting
import nltk.data
nltk.download("popular")   
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [0]:
# to split a review into parsed sentences returning a list of sentences (each sentence is list of words)
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    review = str(review)
    raw_sentences = tokenizer.tokenize(review.strip()) # split para into sentences
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            #sentences.append( review_to_wordlist(raw_sentence,remove_stopwords) )
            sentences.append(raw_sentence) #bcz already cleaned h
    return sentences

In [22]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for r in train:
  sentences += review_to_sentences(r, tokenizer)

print ("Parsing sentences from testing set")
for r in test:
    sentences += review_to_sentences(r, tokenizer)
    
print(sentences[0:5])

Parsing sentences from training set
Parsing sentences from testing set
['guys fuckin crazy stupid shit lick pimply fat ugly bum crazy gays faggots male lesbian know gonna delete stupid block thing care single weirdest boobs okay hate bitches', 'guys victimizing ethnicity blatant racism', 'delete harry walther article worried deletion walther one voices preaching truth jesus message', 'think give shit', 'disagree fact used single party government largely irrelvance important thing still constitutional monarchy parliamentary system allows coalitions majority rule changed say currently coalition whatever party means position deputy prime minister included alongside heads state government false mentioning nick clegg deputy prime minister article around place david cameron currently described prime minister put clegg alongside queen prime minister basis coalition government right job constitutionally defined prime minister second command cameron die resign suddenly clegg would become prime 

In [0]:
#training and saving model 
import logging # for nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [24]:
# Set values for various parameters
num_features = 100    # Word vector dimensionality                      
div_factor = 1000    # division factor for clusters
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

2019-04-19 07:14:20,112 : INFO : collecting all words and their counts
2019-04-19 07:14:20,116 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2019-04-19 07:14:20,123 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2019-04-19 07:14:20,413 : INFO : PROGRESS: at sentence #10000, processed 2216528 words, keeping 27 word types
2019-04-19 07:14:20,715 : INFO : PROGRESS: at sentence #20000, processed 4506219 words, keeping 27 word types
2019-04-19 07:14:21,008 : INFO : PROGRESS: at sentence #30000, processed 6732145 words, keeping 27 word types
2019-04-19 07:14:21,037 : INFO : collected 27 word types from a corpus of 6929356 raw words and 30991 sentences
2019-04-19 07:14:21,038 : INFO : Loading a fresh vocabulary
2019-04-19 07:14:21,039 : INFO : effective_min_count=40 retains 27 unique words (100% of original 27, drops 0)
2019-04-19 07:14:21,040 : INFO : effective_min_count=40 leaves 6929356 word corpus (100% of original 6929356, drops 0)
2019-04-19 07:14:21,041 : INFO : deleting the raw counts dictionary of 27 items
2019-04-19 07:14:21,042 : INFO : sample=0.001 downsamples 23 most-common words
2019-04-19 07:14:21,044 : INFO : downsampling leaves estimated 1194589 word corpus (17.2% of prior 6929356)
2

Converting review to word vectors

In [0]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print ("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, num_features)
       print(reviewFeatureVecs)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [26]:
trainDataVecs = getAvgFeatureVecs( train, model, num_features )
testDataVecs = getAvgFeatureVecs( test, model, num_features )


Output hidden; open in https://colab.research.google.com to view.

Random Forest Classifier

In [27]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(trainDataVecs,train_full[label_name])
pred = clf.predict(testDataVecs)
from sklearn.metrics import classification_report
#test data
print(classification_report(test_full[label_name], pred))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           0       0.59      0.71      0.64      5000
           1       0.71      0.59      0.65      5998

   micro avg       0.65      0.65      0.65     10998
   macro avg       0.65      0.65      0.65     10998
weighted avg       0.66      0.65      0.65     10998



SVM poly

In [28]:
##bullshit for the results
from sklearn.svm import SVC
clf = SVC(kernel='poly')
clf.fit(trainDataVecs,train_full[label_name])
pred = clf.predict(testDataVecs)
from sklearn.metrics import classification_report
#test data
print(classification_report(test_full[label_name], pred))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


              precision    recall  f1-score   support

           0       1.00      0.01      0.02      5000
           1       0.55      1.00      0.71      5998

   micro avg       0.55      0.55      0.55     10998
   macro avg       0.77      0.50      0.36     10998
weighted avg       0.75      0.55      0.39     10998



Clustering these vectors

In [29]:
type(trainDataVecs)
print(trainDataVecs.shape)
tot = len(trainDataVecs)+len(testDataVecs)
print(tot)

arr = np.zeros((tot, num_features))

for col in range(0,num_features):
  for row in range(0, len(trainDataVecs)):
    arr[row] = np.array(trainDataVecs[row])
  
  
print(arr.shape)
l1 = len(trainDataVecs)
l2 = len(testDataVecs)
  
  
for col in range(0,num_features):
  for row in range(0, len(testDataVecs)):
    arr[l1 + row] = np.array(testDataVecs[row])


#word_vectors = np.concatenate(trainDataVecs,testDataVecs)
#print(word_vectors.shape)


(19993, 100)
30991
(30991, 100)


In [30]:
word_vectors = np.matrix(arr)
print(word_vectors.shape)

(30991, 100)


In [31]:
#word_vectors = pd.concat([trainDataVecs,testDataVecs])
#num_clusters = int(len(word_vectors) / div_factor)
#from sklearn.cluster import DBSCAN


num_clusters = 50

# Initalize a k-means object and use it to extract centroids
import time

start = time.time() # Start time


clusterer = KMeans( n_clusters = num_clusters, max_iter=500,  n_init=20)

idx = clusterer.fit_predict( word_vectors )


# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print ("Time taken for K Means clustering: ", elapsed, "seconds.")
print (idx)

Time taken for K Means clustering:  102.76102542877197 seconds.
[43 30 11 ... 32 27 21]


In [32]:
trainVecs = np.matrix(trainDataVecs)
testVecs = np.matrix(testDataVecs)

train_clus = idx[0:l1]
test_clus = idx[l1:l1+l2]

trainVecs = np.c_[trainVecs,train_clus]
testVecs = np.c_[testVecs,test_clus]
train_df = pd.DataFrame(trainVecs)
test_df = pd.DataFrame(testVecs)

tr_df = train_df.rename({num_features:'cluster'}, axis='columns')
test_df = test_df.rename({num_features:'cluster'}, axis='columns')
print(tr_df.shape,len(trainlab),list(tr_df))


tl = pd.DataFrame(data={label_name:trainlab}) #np.zeros([l1,])


#tl=pd.Series(train_full[label_name])

train_df = pd.concat([tr_df,trainlab],axis='columns') #adds extra column for labels


(19993, 101) 19993 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 'cluster']


Training models for each cluster

In [33]:
print(trainVecs.shape,testVecs.shape, list(train_df))

(19993, 101) (10998, 101) [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 'cluster', 'non-toxic']


1. Using probability within cluster to determine toxicity

In [34]:
print(num_clusters)

50


In [35]:
cluster_labels = []
for clus in range(0,num_clusters):
  
  clus_set = train_df [train_df['cluster'] == clus]
  labels = clus_set[label_name]
  count1 = len(clus_set [clus_set[label_name] == 1] ) #counts number of non-toxic rows in cluster
  count0 = len(clus_set [clus_set[label_name] == 0] ) #counts number of toxic rows in cluster
  if(count0 > count1):
    probable_pred = 0
  else:
    probable_pred = 1
  cluster_labels.append(probable_pred)
  
  if(clus%100 == 0):
    print('Cluster no ',clus)
    print('Probable pred', probable_pred)

print(len(cluster_labels))

Cluster no  0
Probable pred 1
50


In [0]:
prediction = []
#print(test_df['cluster'])

for clus_id in test_df['cluster']:

  #print(clus_id)
  prediction.append( cluster_labels[ int(clus_id) ] )
  
  
print(len(prediction),len(test_df))

10998 10998


In [0]:
from sklearn.metrics import classification_report
#test data
print(classification_report(testlab, prediction))

              precision    recall  f1-score   support

           0       0.65      0.60      0.62      5000
           1       0.69      0.73      0.71      5998

   micro avg       0.67      0.67      0.67     10998
   macro avg       0.67      0.67      0.67     10998
weighted avg       0.67      0.67      0.67     10998



Distance-weight implementation

In [0]:
z = np.zeros([l2,1])
preds = pd.DataFrame(data={'preds':[z] })  
test_df = pd.concat([test_df,preds],axis=1) #adds extra column for labels


In [37]:
list(test_df)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 'cluster',
 'preds']

In [0]:
def getEuclideanDist(vec1,vec2):
  dis = 0  
  #print(len(vec1),len(vec2))
  v1 = np.array(vec1)
  v2 = np.array(vec2)

  #print('for euclidean, columns: ',cols)
  
  for field in range(0,num_features):
    dis = dis + np.square( (v1[field] - v2[field]) )
    
  dis = np.sqrt(dis)
  return dis

In [0]:
def compareDistances(clus,testVec):
  
  clus_test = test_df[ test_df['cluster'] == clus ]
  
  
  clus_set = train_df [train_df['cluster'] == clus]
  list(clus_set)
  
  
  zero_wale = clus_set[ clus_set[label_name] == 0 ] #takes all non-toxic = 0 within cluster
  ones_wale = clus_set[ clus_set[label_name] == 1 ] #takes all non-toxic = 1 within cluster
  
  vector0 = zero_wale.drop(columns = [label_name,'cluster'])
  vector1 = ones_wale.drop(columns = [label_name,'cluster'])

  v0_len = len(vector0[0][:])
  v1_len = len(vector1[0][:])
  #print(v0_len,v1_len)
  
  distance0 = 0
  distance1 = 0
  
  for index in range(0,v0_len):
    trainVec = vector0.iloc[index,:]
    #print(trainVec.shape,testVec.shape)
    dist = getEuclideanDist(trainVec,testVec)
    distance0 = distance0 + dist
  
  
  for index in range(0,v1_len):
    trainVec = vector1.iloc[index,:]
    #print(trainVec.shape,testVec.shape)
    dist = getEuclideanDist(trainVec,testVec)
    distance1 = distance1 + dist
    
  if(distance0 > distance1):
    pred = 0
  else:
    pred = 1
    
  return pred
  
 

Predicting by using models per cluster

In [0]:
prediction = []
for j in range(0,l2):
  cluster_id = test_clus[j]
  testVector = test_df.iloc[j,0:num_features]
  #print(list(testVector.shape))
  
  pred = compareDistances(cluster_id, testVector)
  #print(j)
  prediction.append(pred)


In [41]:
print(prediction)

[0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 

In [42]:
len(prediction)

10998

In [43]:
from sklearn.metrics import classification_report
#test data
print(classification_report(testlab, prediction))

              precision    recall  f1-score   support

           0       0.63      0.60      0.62      5000
           1       0.68      0.71      0.69      5998

   micro avg       0.66      0.66      0.66     10998
   macro avg       0.66      0.66      0.66     10998
weighted avg       0.66      0.66      0.66     10998

